# Static Visualization Project  - Democracy Paradox
#### Jose Pajuelo 

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import altair as alt    
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

## 1. Obtaining the data

In [ ]:
DATA_DIR = Path("../data")
dir=DATA_DIR/'iso.csv'
iso=pd.read_csv(dir)

In [ ]:
dir=DATA_DIR/'API_NY.GDP.PCAP.PP.KD_DS2_en_csv_v2_22981.csv'
GDP=pd.read_csv(dir, skiprows=4)
GDP=GDP.drop(columns=['Unnamed: 69'])
GDP_long=GDP.melt(id_vars=['Country Name','Country Code','Indicator Name','Indicator Code'],var_name='Year',value_name='GDP')
GDP_long['Year']=GDP_long['Year'].astype(int)  
GDP=GDP_long[['Country Name', 'Country Code', 'Year', 'GDP']] 
GDP.rename(columns={'GDP':'GDP_per_capita'}, inplace=True)

GDP=pd.merge(GDP, iso, left_on='Country Code', right_on='iso3', how='inner')

In [ ]:
dir=DATA_DIR/'All_data_FIW_2013-2025.xlsx'
Freedom_House=pd.read_excel(dir, sheet_name="FIW13-25", skiprows=1)
Freedom_House=Freedom_House[['Country/Territory', 'Region', 'Edition',  'Status', 'Total']]

#Manual country name adjustments to match other databases
Freedom_House['Country/Territory']=Freedom_House['Country/Territory'].str.replace('Bahamas', 'Bahamas, The')
Freedom_House['Country/Territory']=Freedom_House['Country/Territory'].str.replace('Cabo Verde', 'Cape Verde')
Freedom_House['Country/Territory']=Freedom_House['Country/Territory'].str.replace('Congo (Brazzaville)', 'Republic of the Congo')
Freedom_House['Country/Territory']=Freedom_House['Country/Territory'].str.replace('Congo (Kinshasa)', 'Democratic Republic of the Congo')
Freedom_House['Country/Territory']=Freedom_House['Country/Territory'].str.replace('North Macedonia', 'Macedonia')
Freedom_House['Country/Territory']=Freedom_House['Country/Territory'].str.replace('Slovakia', 'Slovak Republic')

Freedom_House=pd.merge(Freedom_House, iso, left_on='Country/Territory', right_on='country', how='inner')


In [ ]:
dir=DATA_DIR/'p5v2018.xls'
polityv=pd.read_excel(dir,engine='xlrd')
polityv=polityv[['scode', 'country','year','polity2', 'polity', 'durable']]
polityv["year_polity"]=polityv["year"]

#Manual country name adjustments to match other databases
polityv['country']=polityv['country'].str.replace('Bosnia', 'Bosnia and Herzegovina')
polityv['country']=polityv['country'].str.replace('Congo Brazzaville', 'Republic of the Congo' )
polityv['country']=polityv['country'].str.replace('Congo-Brazzaville', 'Republic of the Congo' )
polityv['country']=polityv['country'].str.replace('Congo Kinshasa', 'Democratic Republic of the Congo' )
polityv['country']=polityv['country'].str.replace("Cote D'Ivoire", "Cote d'Ivoire" )
polityv['country']=polityv['country'].str.replace("Gambia", "Gambia, The" )
polityv['country']=polityv['country'].str.replace("Ivory Coast", "Cote d'Ivoire")
polityv['country']=polityv['country'].str.replace("Korea North", "North Korea")
polityv['country']=polityv['country'].str.replace("Korea South", "South Korea")
polityv['country']=polityv['country'].str.replace("Myanmar (Burma)", "Myanmar")
polityv['country']=polityv['country'].str.replace("USSR", "Soviet Union")
polityv['country']=polityv['country'].str.replace("Sudan-North", "Sudan")

polityv=pd.merge(polityv, iso, left_on='country', right_on='country', how='inner')

In [ ]:
#Historic GDP
dir=DATA_DIR/'mpd2023_web.xlsx'
historic_gdp=pd.read_excel(dir, sheet_name="GDPpc")

## 2. Functions

In [ ]:
# Function for write the exponential line
def exp_line(data, x_col, y_col, color_):
    data_ = data[data[x_col].notnull() & data[y_col].notnull()]
    data_ = data_[data_[y_col] > 0]  
    
    #Making the transformation for exponential regression
    y_ln = np.log(data_[y_col])
    x = data_[x_col]
    
    # Regression and obtaning the parameters
    b, log_a = np.polyfit(x, y_ln, 1)
    a = np.exp(log_a)
    y_pred = a * np.exp(b * x) 
    
    df_pred = pd.DataFrame({'x': x, 'y_pred': y_pred})
    line = (alt.Chart(df_pred).mark_line(color=color_)
        .encode(x='x:Q', y='y_pred:Q'))
   
    return line


In [ ]:

#Scatter plot function
def create_scatter_plot(data, x_col, y_col, title_label, x_label, y_label, iso, color_type, color_R, width_l=600, height_l=400):
    chart=alt.Chart(data).mark_circle(size=60, color=color_type).encode(
    x=alt.X(x_col, title=x_label),
    y=alt.Y(y_col, title=y_label)
    ).properties(title=title_label, width=width_l, height=height_l)

    #Adding ISO graphic in same code block
    if iso==True:
        chart_labels = alt.Chart(data).mark_text(
        align='right', 
        baseline='middle',
        fontSize=8 ).encode(
        x=alt.X(x_col, title=x_label), 
        y=alt.Y(y_col, title=y_label), 
        text='iso3'    
        ).properties(title=title_label, 
                     width=width_l, 
                     height=height_l)

        chart_final=chart_labels + exp_line(data, x_col, y_col, color_R)
    else:

        chart_final=chart + exp_line(data, x_col, y_col, color_R)

    return chart_final

#Creating a helperf function for foot notes
def foot_note(foot_note_label, width_l=10, height_l=20, color_l='gray', size_l=10, 
              font_l='sans-serif ', weight_l='normal', aligment_l='left'):
    chart = (
    alt.Chart(pd.DataFrame({'text': [foot_note_label]}))
    .mark_text(align=aligment_l, baseline='bottom',
        dx=0, dy=5,
        size=size_l,
        color=color_l,
        font=font_l,
        fontWeight=weight_l
    )
    .encode(
        text='text:N'
    )
    .properties(
        width=width_l,  
        height=height_l
    ))

    return chart

## 3. Charts

### Graphic 1: Relation between GDP and Democracy

In [ ]:
merged_data=pd.merge(Freedom_House, GDP, left_on=['iso3','Edition'], right_on=['iso3','Year'], how='inner')
merged_data=merged_data[merged_data['Year']==2024]
merged_data['GDP_per_capita']=merged_data['GDP_per_capita'].round(0)
merged_data = merged_data.rename(columns={"Total": "FH_Score"})


In [ ]:
world_plot=create_scatter_plot(merged_data, 
                    'FH_Score', 'GDP_per_capita', 
                    'Figure 1: Relationship Democracy - GDP per capita (2024)', 
                    'Democracy Index (0-100)',['GDP per capita PPP*', '(constant 2021 international $)'],
                    False,
                    "#04724D", "#A53F2B",
                    250, 175) 


foot_label=foot_note(["*PPP: Purchasing Power Parity. Refers to the adjustment of",
                      " income for differences in price levels between countries.",
                      "Source: Freedom House (2025) and World Bank (2025).", 
                      "Each point represents a country"],
                       )


final = alt.vconcat(world_plot, foot_label, spacing=4).configure_view(fill='#efefef').configure(
            background='#efefef').configure_axis(
                labelFont='Times New Roman', titleFontSize=10).configure_title(
                font='Times New Roman', fontSize=14)

final

### Graphic 2: Relationship by regions

In [ ]:

americas_data=merged_data[merged_data['Region']=='Americas']
#americas_data=merged_data[merged_data['Region'].isin(['Americas', 'Europe'])]
europe_data=merged_data[merged_data['Region']=='Europe']    
asia_data=merged_data[merged_data['Region'].isin(['Asia', 'Middle East'])]
americas_plot= create_scatter_plot(americas_data, 
                    'FH_Score', 'GDP_per_capita', 
                    'Americas', 
                    '', ['GDP per capita, PPP' , '(Constant 2021 international $)'],                 
                    True,
                "#04724D", "#A53F2B",
                    200,175)

europe_plot=create_scatter_plot(europe_data, 
                    'FH_Score', 'GDP_per_capita', 
                    'Europe*', 
                    'Democracy Index (0-100)', '', 
                    True,
                "#04724D", "#A53F2B",
                200,175)

asia_plot=create_scatter_plot(asia_data, 
                    'FH_Score', 'GDP_per_capita',  
                    'Asia and Oceania',
                    '', '',
                    True,
              "#04724D", "#A53F2B",
               200,175)

inferior= europe_plot | asia_plot
superior= world_plot | americas_plot

total_graphic=americas_plot | europe_plot | asia_plot


In [ ]:
title=foot_note("Figure 2: Relationship Democracy - GDP per capita by Region (2024)    ",
    font_l='Times New Roman',
    size_l=15,
    weight_l='bold',
    color_l='black',
    aligment_l='center',
    width_l=700)

foot_label=foot_note(["*The Freedom House definition of Europe does not include Eurasia countries: Russia, Armenia, Azerbaijan, Georgia, Kazakhstan, Kyrgyzstan, Moldova", 
                       "Tajikistan, Turkmenistan, Ukraine, and Uzbekistan.",
                       "Source: Freedom House (2025) and World Bank (2025). Each country is represented by its ISO3 code."])

final = alt.vconcat(title, total_graphic, foot_label, spacing=4).configure_view(stroke=None, fill='#efefef').configure(
            background="#efefef").configure_axis(
                labelFont='DM Sans').configure_title(
                font='DM Sans')

final



### Graphic 4: Map, democracy in the world

In [ ]:
map_df=merged_data[['iso3', "FH_Score"]]
map_df.rename(columns={'FH_Score':'value'}, inplace=True)

In [ ]:
#IA: How to create a colorplath map in altair?
#This was particully challenging, because I wasn't able to add a layer of data to the sintaxys published in documentation of altair
#Also, responses of IA did not work, so I had to try and find a jason that matched the requirements (topo_url)
#   
def map(df, title_label, color_1, color_2):
    # TopoJSON with country properties including `name`
    topo_url = "https://cdn.jsdelivr.net/npm/visionscarto-world-atlas/world/110m.json"

    countries = alt.Data(
        url=topo_url,
        format={"type": "topojson", "feature": "countries"}
    )

    base = (
        alt.Chart(countries)
        .mark_geoshape(fill="#e6e6e6", stroke="#aaaaaa", strokeWidth=0.5)
        .project("equalEarth")
        .properties(width=900, height=480, title= title_label)
    )

    # Choropleth layer
    choropleth = (
        alt.Chart(countries)
        .mark_geoshape(stroke="#FFF4F4", strokeWidth=.2)
        .transform_lookup(
            lookup="properties.a3",        
            from_=alt.LookupData(df, key="iso3", fields=["value","iso3"])
        )
        .encode(
            color=alt.condition(
                "datum.value != null",                              
                alt.Color("value:Q", title="Democracy Index",
                            scale=alt.Scale(range=[color_1, color_2])),
                alt.value("#e6e6e6")                              
            ),
            tooltip=[
                alt.Tooltip("properties.name:N", title="Country"),
                alt.Tooltip("iso3:N", title="ISO3"),
                alt.Tooltip("value:Q", title="Value", format=".2f")
            ]
        )
    )

    map=(base + choropleth)

    return map

In [ ]:
world_map=map(map_df, "", "#A53F2B", "#2C7059")

foot_label=foot_note("Source: Freedom House (2025)", size_l=14)

title=foot_note("Figure 4: Level of democracy in the world (2024)", 
                size_l=20, weight_l='bold', 
                font_l='Times New Roman', color_l='black',
                aligment_l='center', width_l=900)

final = alt.vconcat(title, world_map, foot_label, spacing=4).configure(background="#efefef").configure_view(stroke=None)

final

### GRAPHIC 5: Decline of democracy

In [ ]:
mean_by_year = Freedom_House.groupby('Edition')['Total'].mean().reset_index()
foot_label=foot_note("Source: Freedom House (2025)")

mean_by_year=mean_by_year[mean_by_year['Edition'].between(2015, 2024)]
line = alt.Chart(mean_by_year).mark_line(color="#A53F2B").encode(
    x=alt.X('Edition:O', title=''),
    y=alt.Y('Total:Q', title='Democracy Index', scale=alt.Scale(domain=[55, 63]))
    ).properties(width=275, height=200,title=["Figure 5: Average Democracy Score", "in the World (2015-2024)"])

line=line+line.mark_point(shape='circle', color="#A53F2B", size=90, filled=True)


final = alt.vconcat(line, foot_label, spacing=4).configure_view(stroke=None, fill="#efefef").configure(
            background='#efefef').configure_axis(
                labelFont='Times New Roman').configure_title(
                font='Times New Roman', fontSize=13)

final

final

### GRAPHIC 6: Loss of democracy by status (2015-2024)

In [ ]:
#IA: How to calculate the variation between 2 years of a variable in long format 

#Pivot the data to wide format
wide = Freedom_House.pivot_table(
    index=['Country/Territory', 'iso3'],  
    columns='Edition',
    values='Total'
).reset_index()

wide = wide.rename(columns={2015: 'value_2015', 2024: 'value_2024'})

# Compute variation and growth rate
wide['variation'] = wide['value_2024'] - wide['value_2015']
wide['growth'] = (wide['value_2024'] / wide['value_2015'] - 1) * 100

#We keep the status of the initial year to not lost observations
status_2015=Freedom_House[Freedom_House['Edition']==2015][['iso3', 'Status']]

wide_status=pd.merge(wide, status_2015, on='iso3', how='left')

wide=wide_status


In [ ]:
#Calculating mean growth by status
mean_growth = wide.groupby('Status', as_index=False)['growth'].mean()
mean_growth['Status'] = mean_growth['Status'].map({
    'NF': 'Not Free',
    'F': '  Free',
    'PF': ' Partly Free'
})

#Making the bar chart
loss = alt.Chart(mean_growth).mark_bar(color="#A53F2B").encode(
    x=alt.X('Status:N', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('growth:Q', title="Loss (%) in Democracy index (2015-2024)"),
    tooltip=['Status', 'growth']
).properties(
    title=["Figure 6: Loss of Democracy in the last" , " 10 year by Democracy Status (%)"],
    height=200,
    width=275
)

foot_label=foot_note("Source: Freedom House (2025)")

final = alt.vconcat(loss, foot_label, spacing=4).configure_view(stroke=None, fill="#efefef").configure(
            background='#efefef').configure_axis(titleFontSize=10).configure_title(
                font='Times New Roman', fontSize=13)

final

### Graphic 3. Growth Post Democratization

In [ ]:
#Transforming the excel
first_col = historic_gdp.columns[0]
region_row = historic_gdp.loc[historic_gdp[first_col].eq('Region')].iloc[0]
code_row   = historic_gdp.loc[historic_gdp[first_col].eq('year')].iloc[0]   

region_map = region_row.drop(first_col).to_dict()
code_map   = code_row.drop(first_col).to_dict()

data = historic_gdp[~historic_gdp[first_col].isin(['Region', 'year'])].copy()
data = data.rename(columns={first_col: 'year'})

long = data.melt(
    id_vars='year',
    var_name='country',
    value_name='gdp_pc_2011'
)

long['region'] = long['country'].map(region_map)
long['code']   = long['country'].map(code_map)

long['year'] = pd.to_numeric(long['year'], errors='coerce')
long['gdp_pc_2011'] = pd.to_numeric(long['gdp_pc_2011'], errors='coerce')

long = long.sort_values(['country','year']).reset_index(drop=True)

long = long.dropna(subset=['year', 'gdp_pc_2011'])

long.head()

In [ ]:
gdp_historic=long.copy()
gdp_historic['code']=gdp_historic['code'].str.replace('SUN', 'SVU') #make manual change
#Calculating the growth rate by year
gdp_historic["growth_rate"] = gdp_historic.groupby('country')['gdp_pc_2011'].pct_change() * 100

In [ ]:
gdp_historic=pd.merge(gdp_historic, iso, left_on='code', right_on='iso3', how='inner')

In [ ]:
#Sablishing the democratization years:
df = polityv.sort_values(['country', 'year']).copy()

#Keep the years with transition, where -88 is a transition
# and  the next year is democratic (polity2 >=6)
df['is_trans'] = (df['polity'] == -88)    
df['next_is_dem'] = df.groupby('country')['polity2'].shift(-1).ge(6)

events = (df.loc[df['is_trans'] & df['next_is_dem'], ['country','year']]
            .assign(dem_year=lambda x: x['year'] + 1)[['country','dem_year']])

In [ ]:
#There was a years of democratization transition (consequtive years of -88)
#In order to only stablish as democratization year the first of the series
#I performed the following code with IA assistance
#IA: In the following series (upoloaded a sample of the serie of consequtive years) 
# how to only keep the first year of the series for consequtive year of a same country?

d = events.sort_values(['country', 'dem_year'])

keep = (
    d['country'].ne(d['country'].shift()) |           
    d['dem_year'].ne(d['dem_year'].shift() + 1)               
)

events = (
    d[keep]
    .reset_index(drop=True)
)

In [ ]:
#IA: How to create lagged and lead variables?
ks = list(range(-10, 11))
win = (events.assign(key=1)
             .merge(pd.DataFrame({'k': ks, 'key': 1}), on='key')
             .assign(year=lambda x: x['dem_year'] + x['k'])
             [['country','year','k']])

df = polityv.merge(win, on=['country','year'], how='left')

for k in ks:
    df[f'event_{k}'] = (df['k'] == k).astype('Int64')  

df['event_time'] = df['k']  
df['year_event']=df["year"]-1

polityv_events=df.copy()

In [ ]:
polity_gdp=pd.merge(polityv_events, gdp_historic, left_on=['iso3', 'year_event'], right_on=['iso3', 'year'], how='inner')

In [ ]:
#Making the graphic
polity_gdp = polity_gdp.dropna(subset=['event_time', 'growth_rate'])

data=polity_gdp.copy()
data_before=data[data["event_time"]<=0]
data_after=data[data['event_time']>=0]

line_before = alt.Chart(data_before).mark_line(color="#A53F2B").encode(
    x=alt.X('event_time:O', title='Years since Democratization'),
    y=alt.Y('mean(growth_rate):Q', title='Mean GDP growth per capita (2011 USD)'),

).properties(width=300, height=250,
    title="Figure 3: GDP growth before and after Democratization"
)

line_after = alt.Chart(data_after).mark_line(color="#04724D").encode(
    x=alt.X('event_time:O', title='Years since Democratization'),
    y=alt.Y('mean(growth_rate):Q', title='Mean GDP growth per capita (2011 USD)'),
).properties(width=300, height=250,
    title="Graphic 3: GDP growth before and after Democratization"
)

line_before = line_before + line_before.mark_point(shape='triangle-down', 
                                                   color="#A53F2B", size=90, filled=True)

line_after = line_after + line_after.mark_point(shape='triangle-up', 
                                                   color="#04724D", size=90, filled=True)

line=line_before + line_after

vline = alt.Chart(pd.DataFrame({'x': [0]})).mark_rule(
    color='#000000',
    strokeDash=[5, 5],
    size=1
).encode(
    x='x:O'
)

line = line + vline

foot_label=foot_note("Source: Center for Systemic Peace (2018) and Bolt & Van Zanden (2024)")

final = alt.vconcat(line, foot_label, spacing=4).configure(background='white').configure_view(stroke=None)


final = alt.vconcat(line, foot_label, spacing=4).configure_view(stroke=None, fill="#efefef").configure(
            background='#efefef').configure_title(
                font='Times New Roman')

final